In [ ]:
# install libs and python packages
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6 MB 4.3 MB/s 
     |████████████████████████████████| 51 kB 6.0 MB/s 
     |████████████████████████████████| 1.4 MB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 13.4 MB/s 
     |████████████████████████████████| 210 kB 52.4 MB/s 
     |████████████████████████████████| 3.6 MB 43.

In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

ns = music_pb2.NoteSequence()

ns.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
ns.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80)
ns.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80)
ns.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
ns.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
ns.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80)
ns.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80)
ns.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80)
ns.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80)
ns.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80)
ns.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80)
ns.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80)
ns.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
ns.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

# visualize
note_seq.plot_sequence(ns)
note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

In [ ]:
# save as midi
from google.colab import files
note_seq.sequence_proto_to_midi_file(ns, 'star.mid')
files.download('star.mid')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
pg = 5 # instrument
is_dm = False # drum or not
ns2 = music_pb2.NoteSequence()
ns2.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80,program=pg, is_drum=is_dm)
ns2.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80, program=pg, is_drum=is_dm)
ns2.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80, program=pg, is_drum=is_dm) 
note_seq.play_sequence(ns2, synth=note_seq.fluidsynth)

In [ ]:
# Melody RNN

In [ ]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# init model
note_seq.notebook_utils.download_bundle("basic_rnn.mag", "/models/") # download bundle
bundle = sequence_generator_bundle.read_bundle_file("/models/basic_rnn.mag") # load bundle file
generator_map = melody_rnn_sequence_generator.get_generator_map()
melody_rnn = generator_map["basic_rnn"](checkpoint=None, bundle=bundle) # set generator
melody_rnn.initialize() # init


'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /tmp/tmpvy6lyye9/model.ckpt


In [ ]:
# gen sound
from note_seq.protobuf import generator_pb2

base_sequence = ns2
total_time = 36
temperature = 30.2

base_end_time = max(note.end_time for note in base_sequence.notes)

generator_options =  generator_pb2.GeneratorOptions() # gen options
generator_options.args["temperture"].float_value = temperature
generator_options.generate_sections.add(start_time=base_end_time, end_time=total_time)

gen_seq = melody_rnn.generate(base_sequence, generator_options)

note_seq.plot_sequence(gen_seq)
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)

INFO:tensorflow:Beam search yields sequence with log-likelihood: -250.376587 


In [ ]:
#Music VAE

In [ ]:
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/

In [ ]:
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

music_vae = TrainedModel(configs.CONFIG_MAP["cat-mel_2bar_big"],
                         batch_size = 4,
                         checkpoint_dir_or_path="mel_2bar_big.ckpt")

INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


INFO:tensorflow:Restoring parameters from mel_2bar_big.ckpt


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


In [ ]:
tw_tw = music_pb2.NoteSequence()

tw_tw.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
tw_tw.notes.add(pitch=60, start_time=0.4, end_time=0.8, velocity=80)
tw_tw.notes.add(pitch=67, start_time=0.8, end_time=1.2, velocity=80)
tw_tw.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
tw_tw.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
tw_tw.notes.add(pitch=69, start_time=2.0, end_time=2.4, velocity=80)
tw_tw.notes.add(pitch=67, start_time=2.4, end_time=3.2, velocity=80)
tw_tw.notes.add(pitch=65, start_time=3.2, end_time=3.6, velocity=80)
tw_tw.notes.add(pitch=65, start_time=3.6, end_time=4.0, velocity=80)
tw_tw.notes.add(pitch=64, start_time=4.0, end_time=4.4, velocity=80)
tw_tw.notes.add(pitch=64, start_time=4.4, end_time=4.8, velocity=80)
tw_tw.notes.add(pitch=62, start_time=4.8, end_time=5.2, velocity=80)
tw_tw.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
tw_tw.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

tw_tw.total_time = 6.4
tw_tw.tempos.add(qpm=75);

note_seq.plot_sequence(tw_tw)
note_seq.play_sequence(tw_tw, synth=note_seq.fluidsynth)

tw_tw_end = music_pb2.NoteSequence()

tw_tw_end.notes.add(pitch=60, start_time=0.0, end_time=0.4, velocity=80)
tw_tw_end.notes.add(pitch=62, start_time=0.4, end_time=0.8, velocity=80)
tw_tw_end.notes.add(pitch=64, start_time=0.8, end_time=1.2, velocity=80)
tw_tw_end.notes.add(pitch=67, start_time=1.2, end_time=1.6, velocity=80)
tw_tw_end.notes.add(pitch=69, start_time=1.6, end_time=2.0, velocity=80)
tw_tw_end.notes.add(pitch=64, start_time=2.0, end_time=2.4, velocity=80)
tw_tw_end.notes.add(pitch=60, start_time=2.4, end_time=3.2, velocity=80)
tw_tw_end.notes.add(pitch=62, start_time=3.2, end_time=3.6, velocity=80)
tw_tw_end.notes.add(pitch=64, start_time=3.6, end_time=4.0, velocity=80)
tw_tw_end.notes.add(pitch=67, start_time=4.0, end_time=4.4, velocity=80)
tw_tw_end.notes.add(pitch=69, start_time=4.4, end_time=4.8, velocity=80)
tw_tw_end.notes.add(pitch=64, start_time=4.8, end_time=5.2, velocity=80)
tw_tw_end.notes.add(pitch=62, start_time=5.2, end_time=5.6, velocity=80)
tw_tw_end.notes.add(pitch=60, start_time=5.6, end_time=6.4, velocity=80) 

tw_tw_end.total_time = 6.4
tw_tw_end.tempos.add(qpm=75); 

note_seq.plot_sequence(tw_tw_end)
note_seq.play_sequence(tw_tw_end, synth=note_seq.fluidsynth)


In [ ]:
n_seq = 8

gen_seq = music_vae.interpolate(
    tw_tw,
    tw_tw_end,
    num_steps=n_seq,
    length=32
)

interp_seq = note_seq.sequences_lib.concatenate_sequences(gen_seq)

note_seq.plot_sequence(interp_seq)
note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)

In [ ]:
n_seq = 8

generated[1].total_time = 5
generated[2].total_time = 5
generated[1].tempos.add(qpm=75);
generated[2].tempos.add(qpm=75);

gen_seq = music_vae.interpolate(
    generated[1],
    generated[2],
    num_steps=n_seq,
    length=128
)

interp_seq = note_seq.sequences_lib.concatenate_sequences(gen_seq)

note_seq.plot_sequence(interp_seq)
note_seq.play_sequence(interp_seq, synth=note_seq.fluidsynth)

In [ ]:
tmp = generated[2]
note_seq.plot_sequence(tmp)
note_seq.play_sequence(tmp, synth=note_seq.fluidsynth)


In [ ]:
import note_seq
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel

music_vae = TrainedModel(configs.CONFIG_MAP["cat-mel_2bar_big"],
                         batch_size = 4,
                         checkpoint_dir_or_path="mel_2bar_big.ckpt")

generated = music_vae.sample(n=5,
                             length=128,
                             temperature=1.0)

for ns in generated:
    note_seq.plot_sequence(ns)
    note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/usr/local/lib/python3.7/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 4, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/lstm_utils.py:99: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  name=name),
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/core.py:255: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  self._names["W"], [input_size + self._num_units, self._num_units * 4])
/usr/local/lib/python3.7/dist-packages/magenta/contrib/rnn.py:754: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=tf.constant_initializer(0.0))


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Do not call `graph_parents`.
INFO:tensorflow:Restoring parameters from mel_2bar_big.ckpt


/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:199: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))
/usr/local/lib/python3.7/dist-packages/magenta/models/music_vae/base_model.py:205: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  kernel_initializer=tf.random_normal_initializer(stddev=0.001))


In [ ]:
#GANSynth

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

     |████████████████████████████████| 5.6 MB 4.5 MB/s 
     |████████████████████████████████| 51 kB 5.6 MB/s 
     |████████████████████████████████| 1.4 MB 5.2 MB/s 
     |████████████████████████████████| 254 kB 53.2 MB/s 
     |████████████████████████████████| 352 kB 54.1 MB/s 
     |████████████████████████████████| 1.6 MB 30.